# Openminded Pysyft 0.29 MNIST using a CNN code
# Federated Learning Course Gharibim code

# 필요한 라이브러리 불러오기 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from ipynb.fs.full.FLDataset import load_dataset, getImgs, getData

# 사용할 인자 설정

In [2]:
class Arguments():
    def __init__(self):
        self.images = 60000 #총 이미지 수
        self.clients = 10 #총 클라이언트 수
        self.rounds = 5 #모든 클라이언트가 한번씩 학습 한 결과를 1라운드
        self.epochs = 5 #훈련 데이터 전체를 반복하는 횟수
        self.local_batches = 64 #학습 한번에 불러올 데이터 크기
        self.lr = 0.01 #얼마나 빠르게 학습할 것인가?
        self.C = 0.9 #라운드에 얼마나 많은 클라이언트를 사용할 것인가
        self.drop_rate = 0.1
        self.torch_seed = 0 #랜덤 시드 고정
        self.log_interval = 100 #10번의 에포크 마다 학습 결과를 출력하기 위한 인자
        self.iid = 'iid' #iid환경에서의 테스트를 하기 위한 인자
        self.split_size = int(self.images / self.clients) #60000/client 수 즉, 클라이언트 마다 할당되는 데이터의 수 
        self.samples = self.split_size / self.images #논문에서 정의하고 있는 샘플 크기 (nk/n)
        self.use_cuda = False
        self.save_model = False

#args라는 변수에 모두 저장되어있어 이후 코드에서 사용할 수 있음
args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# 클라이언트 생성(워커)
* 클라이언트들이 포함된 배열을 만들 것인데, 그 안에 포함되는 값들을 사전 구조로 묶어줄 것이다. 
* key값은 hook이라는 것을 

In [4]:
hook = sy.TorchHook(torch)
clients = []

#10개의 사전 데이터 형태를 생성하고 value에 VritualWorkder id: 값 client+i 를 생성한다.
for i in range(args.clients):
    clients.append({'fl': sy.virtualWorker(hook, id="client{}".format(i+1))}) 
print(clients)

[{'fl': <VirtualWorker id:client1 #objects:0>}, {'fl': <VirtualWorker id:client2 #objects:0>}, {'fl': <VirtualWorker id:client3 #objects:0>}, {'fl': <VirtualWorker id:client4 #objects:0>}, {'fl': <VirtualWorker id:client5 #objects:0>}, {'fl': <VirtualWorker id:client6 #objects:0>}, {'fl': <VirtualWorker id:client7 #objects:0>}, {'fl': <VirtualWorker id:client8 #objects:0>}, {'fl': <VirtualWorker id:client9 #objects:0>}, {'fl': <VirtualWorker id:client10 #objects:0>}]


In [5]:
# 다운로드 MNIST
#!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
#!tar -zxvf MNIST.tar.gz

--2021-07-19 09:21:21--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
www.di.ens.fr (www.di.ens.fr)을(를) 해석하는 중... 129.199.99.14
접속 www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... 접속됨.
HTTP 요청을 전송했습니다. 응답을 기다리는 중입니다... 302 Found
위치: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [따라감]
--2021-07-19 09:21:22--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
접속 www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... 접속됨.
HTTP 요청을 전송했습니다. 응답을 기다리는 중입니다... 200 OK
길이: 지정되지 않음 [application/x-gzip]
다음 위치에 저장: `MNIST.tar.gz.1'

MNIST.tar.gz.1          [            <=>     ]  17.17M  1.49MB/s               ^C
MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/train-images-idx3-ubyte.gz
MNIST/processed/
MNIST/processed/training.pt
MNIST/processed/test.pt


앞서 생성한 mnist iid 데이터 분할 방법을 사용하여 데이터를 global_train, global_test, train_goup, test_group으로 나눈다.
이후 train_group과 test_group을 활용하여 클라이언트 별 훈련 데이터, 테스트 데이터를 나눠놓는다.

In [6]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)
# 클라이언트 별로 데이터로더를 생성
for idx, client in enumerate(clients):
    trainset_idx_list = list(train_group[idx])
    client['trainset'] = getImgs(global_train, trainset_idx_list, args.local_batches) # 훈련 데이터 로더
    client['testset'] = getImgs(global_test, list(test_group[idx]), args.local_batches) #테스트 데이터 로더
    client['samples'] = len(trainset_idx_list) / args.images #추후 사용할 samples변수 정의

In [7]:
#getImgs하기 전 데이터 모양은 데이터셋 전체이기 때문에, 데이터 로더를 사용하여 전체 테스트 데이터셋 데이터 로더를 생성
global_test_loader = DataLoader(global_test, batch_size=args.local_batches, shuffle=True)

# 학습하려는 모델 정의

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1) #input, output, kernel_size, stride
        
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        #[-1, 50, 8,8]
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # x shape[-1, 20, 24, 24]
        x = F.max_pool2d(x, 2, 2)
        # x shape[-1, 20, 12, 12] 
        x = F.relu(self.conv2(x))
        # x shape[-1, 50, 8, 8]
        x = F.max_pool2d(x, 2, 2)
        # x shape[-1, 50, 4, 4]
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        #x shape [-1, 500]
        x = self.fc2(x)
        #x shape [-1, 10]
        return F.log_softmax(x, dim=1)

In [9]:
def ClientUpdate(args, device, client):
    client['model'].train()
    client['model'].send(client['fl'])
    
    #에포크 수만큼 반복 시키기 위해 범위를 지정했고,
    #출력문 문제 때문에 1부터 시작하였음
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (X, y) in enumerate(client['trainset']):
            #X와 Y를 불러오고 X와 y를 클라이언트에게 전송함
            X = X.send(client['fl'])
            y = y.send(client['fl'])
            X, y = X.to(device), y.to(device)
            #학습 프로세스
            client['optim'].zero_grad() # 그라디언트 초기화
            output = client['model'](X) # 모델의 예측값 획득
            loss = F.nll_loss(output, y) # loss 계산
            loss.backward() #역전파
            client['optim'].step() #파라미터 업데이트
            
            #loss를 출력하기 위한 출력문
            if batch_idx % args.log_interval == 0:
                loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['fl'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
    client['model'].get()

In [10]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        #test 데이터 로더를 불러와서 예측해보기
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            test_loss += F.nll_loss(output, y, reduction='sum').item() #배치 로스 합
            pred = output.argmax(1, keepdim=True) # 결과 값으로 출력되는 log-probability를 클래스 숫자로 변경 [0,0,0,1,0,0,0]
            correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
def averageModels(global_model, clients):
    client_models = [clients[i]['model'] for i in range(len(clients))]
    samples = [clients[i]['samples'] for i in range(len(clients))]
    global_dict = global_model.state_dict()
    
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() * samples[i] for i in range(len(client_models))], 0).sum(0)
            
    global_model.load_state_dict(global_dict)
    return global_model

In [12]:
torch.manual_seed(args.torch_seed) # innitialize w0
global_model = Net() #initialize model

#로컬 클라이언트 모델을 torch cpu에 로드 시키고 최적화함수를 해당 클라이언트 모델로 갱신하는 방식
#clients라는 사전에 모두 저장할 수 있도록 코드를 구성함
for client in clients:
    torch.manual_seed(args.torch_seed) 
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

for fed_round in range(args.rounds):
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # 선택된 클라이언트 집합을 생성하는 방법
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False) #10개의 클라이언트 중 9개의 클라이언트를 선택함
    selected_clients = [clients[i] for i in selected_clients_inds]

    
    # 학습 진행
    # 논문에서 학습 진행에 ClientUpdate함수가 사용되기때문에 이를 구현
    for client in selected_clients:
        ClientUpdate(args, device, client)
    
    # 평균
    global_model = averageModels(global_model, selected_clients)
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

/home/dilab/anaconda3/envs/pysyft/lib/python3.7/site-packages/syft/frameworks/torch/hook/hook.py:560: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)
/home/dilab/anaconda3/envs/pysyft/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:156: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  to_return = self.native_grad


Model client3 Train Epoch: 1 [0/6016 (0%)]	Loss: 2.304334
Model client3 Train Epoch: 2 [0/6016 (0%)]	Loss: 2.264463
Model client3 Train Epoch: 3 [0/6016 (0%)]	Loss: 2.124718
Model client3 Train Epoch: 4 [0/6016 (0%)]	Loss: 1.623853
Model client3 Train Epoch: 5 [0/6016 (0%)]	Loss: 0.717422
Model client9 Train Epoch: 1 [0/6016 (0%)]	Loss: 2.302658
Model client9 Train Epoch: 2 [0/6016 (0%)]	Loss: 2.266213
Model client9 Train Epoch: 3 [0/6016 (0%)]	Loss: 2.177051
Model client9 Train Epoch: 4 [0/6016 (0%)]	Loss: 1.716066
Model client9 Train Epoch: 5 [0/6016 (0%)]	Loss: 0.922652
Model client5 Train Epoch: 1 [0/6016 (0%)]	Loss: 2.307335
Model client5 Train Epoch: 2 [0/6016 (0%)]	Loss: 2.266932
Model client5 Train Epoch: 3 [0/6016 (0%)]	Loss: 2.128643
Model client5 Train Epoch: 4 [0/6016 (0%)]	Loss: 1.619649
Model client5 Train Epoch: 5 [0/6016 (0%)]	Loss: 0.693400
Model client10 Train Epoch: 1 [0/6016 (0%)]	Loss: 2.297128
Model client10 Train Epoch: 2 [0/6016 (0%)]	Loss: 2.268785
Model client

Model client6 Train Epoch: 3 [0/6016 (0%)]	Loss: 0.160188
Model client6 Train Epoch: 4 [0/6016 (0%)]	Loss: 0.178211
Model client6 Train Epoch: 5 [0/6016 (0%)]	Loss: 0.184034
Model client5 Train Epoch: 1 [0/6016 (0%)]	Loss: 0.313504
Model client5 Train Epoch: 2 [0/6016 (0%)]	Loss: 0.270600
Model client5 Train Epoch: 3 [0/6016 (0%)]	Loss: 0.235557
Model client5 Train Epoch: 4 [0/6016 (0%)]	Loss: 0.178778
Model client5 Train Epoch: 5 [0/6016 (0%)]	Loss: 0.228301
Model client2 Train Epoch: 1 [0/6016 (0%)]	Loss: 0.223734
Model client2 Train Epoch: 2 [0/6016 (0%)]	Loss: 0.351724
Model client2 Train Epoch: 3 [0/6016 (0%)]	Loss: 0.231041
Model client2 Train Epoch: 4 [0/6016 (0%)]	Loss: 0.302091
Model client2 Train Epoch: 5 [0/6016 (0%)]	Loss: 0.183283
Model client3 Train Epoch: 1 [0/6016 (0%)]	Loss: 0.316389
Model client3 Train Epoch: 2 [0/6016 (0%)]	Loss: 0.216821
Model client3 Train Epoch: 3 [0/6016 (0%)]	Loss: 0.250107
Model client3 Train Epoch: 4 [0/6016 (0%)]	Loss: 0.225035
Model client3 